In [ ]:
import tensorflow
import numpy 
import pandas
import nltk
import random
import json
import pickle

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
with open('/content/drive/MyDrive/AI_materials/dataset.json') as jsonfile:
  intents = json.load(jsonfile)
# intents = json.loads(open('dataset.json').read())

In [ ]:
intents

In [ ]:
nltk.download('punkt')

In [ ]:
words = []
classes =[]
documents=[]
ignore_letters = [ '!', ',','.', '?']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [ ]:
# words

In [ ]:
classes = sorted(set(classes))

In [ ]:
pickle.dump(words, open('words.pkl', 'wb'))

In [ ]:
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
output_empty = [0]*len(classes)

In [ ]:
for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)


  output_row = list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = numpy.array(training)

In [ ]:
train_x = list(training[:,0])
train_y = list(training[:, 1])

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(100, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(75, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(numpy.array(train_x), numpy.array(train_y), epochs=30, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)

In [ ]:
model.summary()

# Model is trained and is stored in drive file with the name of chatbotmodel.h5
- Mount drive and import the model using pickle

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
# intents = json.loads(open('/content/drive/MyDrive/AI_materials/dataset.json').read())  # to retrieve from the drive 
intents = json.loads(open('dataset.json').read()) # to read directly from the same folder

In [ ]:
hist

In [ ]:
model = load_model('chatbotmodel.h5')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [ ]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1

  return numpy.array(bag)

In [ ]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(numpy.array([bow]))[0]
  ERROR_THRESHOLD = 0.21
  results = [[i, r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
  return return_list

In [ ]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']

  default_tag = ""
  flag = 1
  for i in list_of_intents:
    if i['tag'] == 'error' and flag==1:
      default_tag = random.choice(i['responses'])
      result = default_tag
      flag = 0
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [ ]:
def bot(): 
  print("Ask Query ......................")
  while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
      ints = predict_class(message)
      res =get_response(ints, intents)
      print("| Bot: ", res)
      print("The program ends here!")
      exit()
      return
    else:
      ints = predict_class(message)
      res= get_response(ints, intents)
      print("| Bot: ", res)

In [ ]:
bot()
